In [1]:
# =============================================================================
# STEP 1: INSTALL REQUIRED DEPENDENCIES
# =============================================================================
# Run this cell first to install all required packages

print("🚀 Starting Healthcare Policy Advisor Setup...")
print("=" * 60)

# Install required packages
!pip install google-generativeai --quiet
!pip install gradio --quiet

print("✅ Dependencies installed successfully!")
print("📋 Installed packages:")
print("   - google-generativeai (for AI responses)")
print("   - gradio (for web interface)")
print("\n🔄 Ready for Step 2: Import libraries and configure API")

🚀 Starting Healthcare Policy Advisor Setup...
✅ Dependencies installed successfully!
📋 Installed packages:
   - google-generativeai (for AI responses)
   - gradio (for web interface)

🔄 Ready for Step 2: Import libraries and configure API


In [2]:
# =============================================================================
# STEP 2: IMPORT LIBRARIES AND CONFIGURE API
# =============================================================================

import google.generativeai as genai
import gradio as gr
from typing import Dict, List, Optional
import json
import re

print("📚 Libraries imported successfully!")
print("=" * 60)

# Configure Google Gemini API
print("🔑 Setting up Google Gemini API...")

# Get API key from Colab secrets
try:
    from google.colab import userdata
    api_key = userdata.get('GOOGLE_API_KEY')

    if not api_key:
        print("❌ No API key found in Colab secrets!")
        print("📝 Please add your Google API key to Colab secrets:")
        print("   1. Click the 🔑 key icon in the left sidebar")
        print("   2. Add a new secret with name: GOOGLE_API_KEY")
        print("   3. Paste your Google API key as the value")
        print("   4. Re-run this cell")
    else:
        genai.configure(api_key=api_key)

        # Test the API connection
        model = genai.GenerativeModel('gemini-1.5-flash')
        test_response = model.generate_content("Test connection")

        print("✅ Google Gemini API configured and tested successfully!")
        print("🔄 Ready for Step 3: Create insurance policy database")

except ImportError:
    print("⚠️  Running outside Colab environment")
    print("📝 Please manually set your API key:")
    print("   api_key = 'your-google-api-key-here'")
    print("   genai.configure(api_key=api_key)")
except Exception as e:
    print(f"❌ API setup failed: {e}")
    print("Please check your API key and try again")

📚 Libraries imported successfully!
🔑 Setting up Google Gemini API...
✅ Google Gemini API configured and tested successfully!
🔄 Ready for Step 3: Create insurance policy database


In [3]:
# =============================================================================
# STEP 3: CREATE INSURANCE POLICY DATABASE
# =============================================================================

print("🏥 Creating Healthcare Insurance Policy Database...")
print("=" * 60)

# Insurance Plans Database
INSURANCE_PLANS = {
    "Essential Care Plan": {
        "monthly_cost": 120,
        "description": "Budget-friendly plan for basic healthcare needs",
        "coverage_items": [
            "Emergency hospital care",
            "Basic doctor consultations",
            "Essential diagnostic tests",
            "Emergency prescription coverage"
        ],
        "annual_limit": 75000,
        "deductible": 2000,
        "age_range": "18-60",
        "network_size": "1,800+ hospitals",
        "best_for": ["budget-conscious", "young adults", "basic needs"]
    },

    "Complete Family Care": {
        "monthly_cost": 280,
        "description": "Comprehensive coverage for entire family",
        "coverage_items": [
            "Full hospitalization coverage",
            "Regular doctor visits",
            "Specialist consultations",
            "Laboratory and diagnostic services",
            "Prescription drug benefits",
            "Emergency ambulance service"
        ],
        "annual_limit": 400000,
        "deductible": 1500,
        "age_range": "All ages, includes dependents",
        "network_size": "4,200+ hospitals",
        "best_for": ["families", "comprehensive care", "children"]
    },

    "Premium Wellness Plan": {
        "monthly_cost": 420,
        "description": "Complete health and wellness package",
        "coverage_items": [
            "All Complete Family Care benefits",
            "Preventive health screenings",
            "Mental health counseling",
            "Wellness and fitness programs",
            "Alternative therapy coverage",
            "Extended prescription benefits"
        ],
        "annual_limit": 800000,
        "deductible": 1000,
        "age_range": "All ages",
        "network_size": "6,500+ hospitals and clinics",
        "best_for": ["wellness focus", "preventive care", "premium coverage"]
    },

    "Senior Care Plus": {
        "monthly_cost": 480,
        "description": "Specialized healthcare for mature adults",
        "coverage_items": [
            "Comprehensive hospitalization",
            "Unlimited specialist visits",
            "Chronic condition management",
            "Regular medication coverage",
            "Vision and hearing benefits",
            "Home healthcare services"
        ],
        "annual_limit": 600000,
        "deductible": 800,
        "age_range": "50+ years",
        "network_size": "3,800+ senior-friendly facilities",
        "best_for": ["seniors", "chronic conditions", "specialized care"]
    }
}

# Additional Coverage Options
EXTRA_COVERAGE = {
    "Dental Care Package": {
        "monthly_cost": 35,
        "includes": ["Routine cleanings", "Basic dental procedures", "Annual check-ups"]
    },
    "Vision Care Package": {
        "monthly_cost": 25,
        "includes": ["Eye exams", "Prescription glasses", "Contact lens coverage"]
    },
    "Maternity Care Add-on": {
        "monthly_cost": 60,
        "includes": ["Prenatal care", "Delivery coverage", "Postnatal support"],
        "waiting_period": "10 months"
    },
    "International Coverage": {
        "monthly_cost": 45,
        "includes": ["Emergency care abroad", "Medical evacuation", "24/7 assistance"]
    }
}

print(f"✅ Created {len(INSURANCE_PLANS)} main insurance plans")
print(f"✅ Created {len(EXTRA_COVERAGE)} additional coverage options")
print("\n📋 Available Plans:")
for plan_name, details in INSURANCE_PLANS.items():
    print(f"   • {plan_name}: ${details['monthly_cost']}/month")

print("\n🔄 Ready for Step 4: Create the AI advisor system")

🏥 Creating Healthcare Insurance Policy Database...
✅ Created 4 main insurance plans
✅ Created 4 additional coverage options

📋 Available Plans:
   • Essential Care Plan: $120/month
   • Complete Family Care: $280/month
   • Premium Wellness Plan: $420/month
   • Senior Care Plus: $480/month

🔄 Ready for Step 4: Create the AI advisor system


In [4]:
# =============================================================================
# STEP 4: CREATE AI ADVISOR SYSTEM
# =============================================================================

print("🤖 Creating AI Healthcare Policy Advisor...")
print("=" * 60)

class HealthcareAdvisor:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        self.conversation_state = {
            "user_profile": {},
            "recommended_plans": [],
            "current_step": "initial"
        }

    def extract_user_info(self, user_message: str) -> dict:
        """Extract key information from user input"""

        extraction_prompt = f"""
        Analyze this user message and extract relevant healthcare insurance information:

        User Message: "{user_message}"

        Extract and return ONLY the following information if mentioned:
        - Age or age range
        - Family size (single, couple, family with children)
        - Number of dependents
        - Budget range or preferences
        - Specific health needs or concerns
        - Special requirements (dental, vision, maternity, etc.)
        - Current health status

        Return the information in a clear, structured format. If something is not mentioned, don't make assumptions.
        """

        try:
            response = self.model.generate_content(extraction_prompt)
            return response.text
        except Exception as e:
            return f"Error processing request: {str(e)}"

    def recommend_best_plan(self, user_info: str) -> str:
        """Generate personalized plan recommendations"""

        # Convert plans to readable format
        plans_info = ""
        for plan_name, details in INSURANCE_PLANS.items():
            plans_info += f"\n{plan_name}:\n"
            plans_info += f"  - Cost: ${details['monthly_cost']}/month\n"
            plans_info += f"  - Coverage Limit: ${details['annual_limit']:,}\n"
            plans_info += f"  - Age Range: {details['age_range']}\n"
            plans_info += f"  - Best For: {', '.join(details['best_for'])}\n"
            plans_info += f"  - Key Benefits: {', '.join(details['coverage_items'][:3])}\n"

        recommendation_prompt = f"""
        You are a healthcare insurance advisor. Based on the user's information, recommend the most suitable insurance plan.

        User Information: {user_info}

        Available Insurance Plans: {plans_info}

        Provide a personalized recommendation that includes:
        1. Primary recommended plan with clear reasoning
        2. Why this plan fits their specific needs
        3. Key benefits that match their requirements
        4. Cost breakdown and value proposition
        5. Alternative plan option if applicable
        6. Suggested add-ons if relevant

        Be conversational, helpful, and focus on matching their specific needs.
        Don't just list features - explain how the plan solves their problems.
        """

        try:
            response = self.model.generate_content(recommendation_prompt)
            return response.text
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"

    def answer_followup(self, question: str, previous_context: str) -> str:
        """Handle follow-up questions about policies"""

        followup_prompt = f"""
        A user is asking a follow-up question about healthcare insurance.

        Previous conversation context: {previous_context}

        User's question: "{question}"

        Available plan details: {json.dumps(INSURANCE_PLANS, indent=2)}

        Additional coverage options: {json.dumps(EXTRA_COVERAGE, indent=2)}

        Provide a helpful, specific answer based on our insurance offerings.
        Include relevant details about coverage, costs, or benefits as needed.
        """

        try:
            response = self.model.generate_content(followup_prompt)
            return response.text
        except Exception as e:
            return f"I apologize, but I'm having trouble processing your question: {str(e)}"

# Initialize the advisor
advisor = HealthcareAdvisor()

print("✅ Healthcare Advisor AI system created successfully!")
print("🔄 Ready for Step 5: Create the web interface")

🤖 Creating AI Healthcare Policy Advisor...
✅ Healthcare Advisor AI system created successfully!
🔄 Ready for Step 5: Create the web interface


In [5]:
# =============================================================================
# STEP 5: CREATE CUSTOM WEB INTERFACE
# =============================================================================

print("🎨 Creating Custom Web Interface...")
print("=" * 60)

def process_user_request(message, chat_history):
    """Main function to process user requests"""

    if not message.strip():
        return "Please share your healthcare insurance needs so I can help you find the perfect plan!"

    # Check if this is a follow-up question
    is_followup = len(chat_history) > 0 and any(
        keyword in message.lower()
        for keyword in ['more', 'about', 'cost', 'cover', 'benefit', 'compare', 'difference']
    )

    if is_followup and chat_history:
        # Handle follow-up question
        last_response = chat_history[-1][1] if chat_history else ""
        response = advisor.answer_followup(message, last_response)
    else:
        # New recommendation request
        user_info = advisor.extract_user_info(message)
        response = advisor.recommend_best_plan(user_info)

    return response

# Custom CSS for modern design
custom_css = """
#main-container {
    max-width: 900px !important;
    margin: 0 auto !important;
}

.gradio-container {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    min-height: 100vh;
}

#chat-container {
    background: white !important;
    border-radius: 15px !important;
    box-shadow: 0 10px 30px rgba(0,0,0,0.2) !important;
    margin: 20px !important;
    padding: 20px !important;
}

.message {
    padding: 12px 16px !important;
    margin: 8px 0 !important;
    border-radius: 12px !important;
}

.user-message {
    background: #e3f2fd !important;
    border-left: 4px solid #2196f3 !important;
}

.bot-message {
    background: #f1f8e9 !important;
    border-left: 4px solid #4caf50 !important;
}

#title-header {
    text-align: center !important;
    color: white !important;
    margin-bottom: 20px !important;
}

#input-box {
    border-radius: 25px !important;
    border: 2px solid #e0e0e0 !important;
    padding: 12px 20px !important;
}

#send-button {
    background: linear-gradient(45deg, #4caf50, #45a049) !important;
    border: none !important;
    border-radius: 20px !important;
    color: white !important;
    padding: 10px 20px !important;
}
"""

# Create the interface
def create_healthcare_interface():

    with gr.Blocks(css=custom_css, title="Healthcare Policy Advisor") as interface:

        # Header
        gr.HTML("""
        <div id="title-header">
            <h1>🏥 Healthcare Insurance Advisor</h1>
            <p>Find Your Perfect Insurance Plan with AI-Powered Recommendations</p>
        </div>
        """)

        with gr.Column(elem_id="main-container"):
            with gr.Column(elem_id="chat-container"):

                # Information panel
                gr.Markdown("""
                ### 💡 How it works:
                1. **Tell me about yourself** - Age, family size, health needs
                2. **Share your priorities** - Budget, specific coverage needs
                3. **Get personalized recommendations** - Tailored to your situation
                4. **Ask follow-up questions** - Get detailed information about any plan

                ### 📋 Our Plans Range: $120 - $480/month
                Start by describing your healthcare insurance needs below!
                """)

                # Chat interface
                chatbot = gr.Chatbot(
                    label="Insurance Advisor Chat",
                    height=400,
                    elem_classes=["chat-messages"]
                )

                # Input area
                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="Describe your healthcare needs (e.g., 'I'm 30, single, need basic coverage under $200/month')",
                        label="Your Message",
                        elem_id="input-box",
                        scale=4
                    )
                    send_btn = gr.Button("Send", elem_id="send-button", scale=1)

                # Clear button
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")

        # Event handlers
        def respond(message, chat_history):
            bot_response = process_user_request(message, chat_history)
            chat_history.append((message, bot_response))
            return "", chat_history

        # Button events
        send_btn.click(respond, [msg_input, chatbot], [msg_input, chatbot])
        msg_input.submit(respond, [msg_input, chatbot], [msg_input, chatbot])
        clear_btn.click(lambda: [], None, chatbot)

    return interface

# Create the interface
healthcare_app = create_healthcare_interface()

print("✅ Custom web interface created successfully!")
print("🎨 Features included:")
print("   • Modern gradient design")
print("   • Responsive chat interface")
print("   • Custom styling and colors")
print("   • Clear conversation flow")
print("\n🔄 Ready for Step 6: Launch the application")

🎨 Creating Custom Web Interface...


<ipython-input-5-129886237>:117: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


✅ Custom web interface created successfully!
🎨 Features included:
   • Modern gradient design
   • Responsive chat interface
   • Custom styling and colors
   • Clear conversation flow

🔄 Ready for Step 6: Launch the application


In [ ]:
# =============================================================================
# STEP 6: LAUNCH THE HEALTHCARE POLICY ADVISOR
# =============================================================================

print("🚀 Launching Healthcare Policy Advisor...")
print("=" * 60)

# Launch the application
try:
    # Launch with public sharing enabled
    healthcare_app.launch(
        share=True,           # Creates shareable public link
        debug=True,           # Enable debug mode for development
        server_name="0.0.0.0", # Allow external connections
        server_port=7860,     # Default port
        inbrowser=True        # Open in browser automatically
    )

    print("✅ Application launched successfully!")
    print("🌐 Your app is now running with a public shareable link")
    print("📱 The interface will work on both desktop and mobile devices")

except Exception as e:
    print(f"❌ Launch failed: {e}")
    print("💡 Try running the previous steps again if you encounter issues")

print("\n" + "=" * 60)
print("🎉 SETUP COMPLETE!")
print("=" * 60)
print("Your Healthcare Policy Advisor is now ready to use!")
print("\n📋 What you can do:")
print("• Get personalized insurance recommendations")
print("• Compare different plans based on your needs")
print("• Ask specific questions about coverage and costs")
print("• Explore add-on options for additional coverage")
print("\n💬 Example conversations to try:")
print("• 'I'm 25, single, looking for affordable basic coverage'")
print("• 'Family of 4, need comprehensive plan with dental coverage'")
print("• 'Senior citizen needing specialized care coverage'")
print("• 'Expecting a baby, need maternity benefits'")

print("\n🔗 Use the public link above to share with others!")
print("⚠️  Note: The public link will expire in 1 week")

🚀 Launching Healthcare Policy Advisor...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://29ddf06043977814f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
